#Overview
The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program.

In [ ]:
! pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


# Normalize pixel values between 0 and 1

In [ ]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Get the optimal hyperparameters

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



# Build the model with the optimal hyperparameters and train it on the data for 50 epochs

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.5031 - accuracy: 0.8207 - val_loss: 0.4301 - val_accuracy: 0.8443
Epoch 2/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3736 - accuracy: 0.8649 - val_loss: 0.3585 - val_accuracy: 0.8724
Epoch 3/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3352 - accuracy: 0.8779 - val_loss: 0.3558 - val_accuracy: 0.8668
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3096 - accuracy: 0.8859 - val_loss: 0.3562 - val_accuracy: 0.8720
Epoch 5/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2913 - accuracy: 0.8918 - val_loss: 0.3369 - val_accuracy: 0.8797
Epoch 6/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2741 - accuracy: 0.8974 - val_loss: 0.3301 - val_accuracy: 0.8846
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2618 - accuracy: 0.9026 - val_loss: 0.3164 - val_accurac

# Retrain the model

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/41
1500/1500 [==============================] - 9s 6ms/step - loss: 0.5081 - accuracy: 0.8209 - val_loss: 0.4077 - val_accuracy: 0.8584
Epoch 2/41
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3764 - accuracy: 0.8628 - val_loss: 0.3922 - val_accuracy: 0.8583
Epoch 3/41
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3355 - accuracy: 0.8765 - val_loss: 0.3554 - val_accuracy: 0.8743
Epoch 4/41
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3115 - accuracy: 0.8842 - val_loss: 0.3923 - val_accuracy: 0.8482
Epoch 5/41
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2914 - accuracy: 0.8922 - val_loss: 0.3424 - val_accuracy: 0.8797
Epoch 6/41
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2772 - accuracy: 0.8961 - val_loss: 0.3207 - val_accuracy: 0.8833
Epoch 7/41
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2637 - accuracy: 0.9015 - val_loss: 0.3352 - val_accuracy:

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.5000 - accuracy: 0.8911
[test loss, test accuracy]: [0.5000040531158447, 0.8910999894142151]
